In [1]:
import pandas as pd 
from sqlalchemy import create_engine
from binance.client import Client

In [2]:
# Binance API keys
API_KEY = '3KKoPw7y0uHyLzFVnwxu6gsXFQfBEDwnoLDpKjbh41UgvbB3LukYx0zhsG5l1xev'
API_SECRET = 'PmJwK0lzZiQ5cCEW2aypB8Eska7bHiJ3gmOY6WyetmTm7C7FxCKBkfvYJR3Wzhky'

In [3]:
client = Client(API_KEY,API_SECRET)

In [4]:
# Fetch live candlestick data
symbol = "ADAUSDT"
interval = Client.KLINE_INTERVAL_1MINUTE  # 1-minute interval

In [5]:
# Get historical klines (candlesticks)
candlesticks = client.get_klines(symbol=symbol, interval=interval, limit=100)

In [6]:
columns = [
    "Open Time", "Open", "High", "Low", "Close", "Volume", "Close Time",
    "Quote Asset Volume", "Number of Trades", "Taker Buy Base Volume",
    "Taker Buy Quote Volume", "Ignore"
]
df = pd.DataFrame(candlesticks, columns=columns)


In [7]:
# Convert to DataFrame

# Convert timestamps to readable format
df["Open Time"] = pd.to_datetime(df["Open Time"], unit="ms")
df["Close Time"] = pd.to_datetime(df["Close Time"], unit="ms")

print(df.head())

            Open Time        Open        High         Low       Close  \
0 2024-11-25 13:48:00  1.01530000  1.01780000  1.01520000  1.01740000   
1 2024-11-25 13:49:00  1.01720000  1.01750000  1.01360000  1.01520000   
2 2024-11-25 13:50:00  1.01520000  1.01590000  1.01410000  1.01590000   
3 2024-11-25 13:51:00  1.01600000  1.02020000  1.01490000  1.01980000   
4 2024-11-25 13:52:00  1.01960000  1.02010000  1.01840000  1.01890000   

            Volume              Close Time Quote Asset Volume  \
0  189027.00000000 2024-11-25 13:48:59.999    192117.83858000   
1  213363.90000000 2024-11-25 13:49:59.999    216642.71106000   
2  178062.10000000 2024-11-25 13:50:59.999    180690.89170000   
3  209277.80000000 2024-11-25 13:51:59.999    213034.64315000   
4  187238.80000000 2024-11-25 13:52:59.999    190849.88134000   

   Number of Trades Taker Buy Base Volume Taker Buy Quote Volume Ignore  
0               391        62420.70000000         63464.51549000      0  
1               653   

In [8]:
# Create SQLite engine
engine = create_engine('sqlite:///binance_data.db')

In [9]:
# Save DataFrame to SQL table
table_name = "adausdt_candlesticks"
df.to_sql(table_name, engine, if_exists="replace", index=False)

print(f"Data stored in table '{table_name}'.")

Data stored in table 'adausdt_candlesticks'.


In [10]:
# Query data using SQL
query = f"SELECT * FROM {table_name} WHERE Volume > 1000"

# Fetch data from SQLite
filtered_df = pd.read_sql(query, engine)
print(filtered_df)

                     Open Time        Open        High         Low  \
0   2024-11-25 13:48:00.000000  1.01530000  1.01780000  1.01520000   
1   2024-11-25 13:49:00.000000  1.01720000  1.01750000  1.01360000   
2   2024-11-25 13:50:00.000000  1.01520000  1.01590000  1.01410000   
3   2024-11-25 13:51:00.000000  1.01600000  1.02020000  1.01490000   
4   2024-11-25 13:52:00.000000  1.01960000  1.02010000  1.01840000   
..                         ...         ...         ...         ...   
95  2024-11-25 15:23:00.000000  0.99570000  0.99870000  0.99340000   
96  2024-11-25 15:24:00.000000  0.99850000  1.00120000  0.99820000   
97  2024-11-25 15:25:00.000000  1.00110000  1.00280000  1.00050000   
98  2024-11-25 15:26:00.000000  1.00230000  1.00510000  1.00080000   
99  2024-11-25 15:27:00.000000  1.00350000  1.00370000  1.00230000   

         Close           Volume                  Close Time  \
0   1.01740000  189027.00000000  2024-11-25 13:48:59.999000   
1   1.01520000  213363.90000000  